In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM


df = pd.read_excel('/Users/martinwolf/Python/Superdepot Reporting/Data/df.xlsx')
df['Pick Datum'] = pd.to_datetime(df['B']).dt.date


In [ ]:
def DatenKundeaufbereiten(dfa):
    # DataFrame Picks zu Kunden
    dfapicks = dfa.groupby(['Lieferschein','Pick Datum','Kunde'],dropna =False)['Picks Gesamt'].sum().reset_index()
    dfapicksinCS = dfa.groupby(['Lieferschein','Pick Datum'])['Picks CS'].sum().reset_index()
    dfapicksinPAL = dfa.groupby(['Lieferschein','Pick Datum'])['Picks PAL'].sum().reset_index()
    dfapicksinOUT = dfa.groupby(['Lieferschein','Pick Datum'])['Picks OUT'].sum().reset_index()
    dfakunden = pd.merge(dfapicks, dfapicksinCS[['Lieferschein','Picks CS']],left_on='Lieferschein', right_on='Lieferschein',how='left')
    dfakunden = pd.merge(dfakunden, dfapicksinOUT[['Lieferschein','Picks OUT']],left_on='Lieferschein', right_on='Lieferschein',how='left')
    dfakunden = pd.merge(dfakunden, dfapicksinPAL[['Lieferschein','Picks PAL']],left_on='Lieferschein', right_on='Lieferschein',how='left')
    # DataFrame Picks zu Tag
    return dfakunden
def DatenPickTagaufbereiten(dfT):
    dfaPicksTag = dfT.groupby(['Pick Datum'])['Picks Gesamt'].sum().reset_index()
    dfaCStag = dfT.groupby(['Pick Datum'])['Picks CS'].sum().reset_index()   
    dfaPALtag = dfT.groupby(['Pick Datum'])['Picks PAL'].sum().reset_index()      
    dfaOUTtag = dfT.groupby(['Pick Datum'])['Picks OUT'].sum().reset_index()
    dfaZugriffe = pd.merge(dfaPicksTag, dfaCStag[['Pick Datum','Picks CS']],left_on='Pick Datum', right_on='Pick Datum',how='left')         
    dfaZugriffe = pd.merge(dfaZugriffe, dfaOUTtag[['Pick Datum','Picks OUT']],left_on='Pick Datum', right_on='Pick Datum',how='left')
    dfaZugriffe = pd.merge(dfaZugriffe, dfaPALtag[['Pick Datum','Picks PAL']],left_on='Pick Datum', right_on='Pick Datum',how='left')
    return dfaZugriffe

In [ ]:
dfPicks = DatenPickTagaufbereiten(df)

df = dfPicks[['Pick Datum','Picks Gesamt']]
df.index = df['Pick Datum']
del df['Pick Datum']
testdays = 60

bitcoin = df.filter(['Picks Gesamt'])

In [ ]:
bitcoin_data = bitcoin.values
bitcoin_data

In [ ]:
import math


training_len = math.ceil(len(bitcoin) * 0.8)


In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(bitcoin_data)
